In [ ]:
%%HTML
<style>
.container{width:90%}
</style>
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>

In [ ]:
import ipywidgets as w
from IPython.display import display,clear_output
import random
import time

class Character():
    def __init__(self):
        self.location = [0,0]
        self.max_hp = 100
        self.max_mana = 100
        self.set_bars()
        self.attacks = {
            'fireball': {'power':15,'max_uses':10,'uses_left':10},
            'tackle' : {'power':10,'max_uses':30,'uses_left':30}
        }
    def set_bars(self):
        self.hp_bar = w.FloatProgress(value = self.max_hp, 
                                 min = 0, 
                                 max = self.max_hp,
                                 style={'bar_color': 'green','background':'black'},
                                 layout=w.Layout(display="flex", justify_content="flex-start")
                                )
        self.mana_bar = w.FloatProgress(value = self.max_mana,
                                   min = 0, 
                                   max = self.max_mana,
                                   style={'bar_color': 'blue','background':'black'},
                                   layout=w.Layout(display="flex", justify_content="flex-start")                                    
                                  )
    def remove_hp(self, damage):
        self.hp_bar.value = max(0, self.hp_bar.value-damage)
    def remove_mana(self, cost):
        self.mana_bar.value = max(0, self.mana_bar.value-cost)
    def draw_bars(self):
        display(w.VBox([self.hp_bar,self.mana_bar]))
        
class Enemy():
    def __init__(self):
        self.max_hp = 100
        self.max_mana = 100
        self.set_bars()
    def set_bars(self):
        self.hp_bar = w.FloatProgress(value = self.max_hp, 
                                 min = 0, 
                                 max = self.max_hp,
                                 style={'bar_color': 'red','background':'black'},
                                 layout=w.Layout(display="flex", justify_content="flex-end")
                                )
        self.hp_label = w.Label(str(int(self.hp_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-end"))
        self.mana_bar = w.FloatProgress(value = self.max_mana,
                                   min = 0, 
                                   max = self.max_mana,
                                   style={'bar_color': 'blue','background':'black'}, 
                                   layout=w.Layout(display="flex", justify_content="flex-end")
                                  )
        self.mana_label = w.Label(str(int(self.mana_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-end"))
    def remove_hp(self, damage):
        self.hp_bar.value = max(0, self.hp_bar.value-damage)
        self.hp_label.value = str(int(self.hp_bar.value))
    def remove_mana(self, cost):
        self.mana_bar.value = max(0, self.mana_bar.value-cost)
        self.mana_label.value = str(int(self.mana_bar.value))
    def draw_bars(self):
        display(w.VBox([self.hp_bar,self.mana_bar]))
        
class interactions():
    def __init__(self):
        
        self._ability_damage_ = {
                'tackle':40,
                'scratch':50
            }
    def attack(self,target_entity,ability):
        target_entity.remove_hp(self._ability_damage_[ability])
        
class Overworld():
    def __init__(self, map_array, player):
        self.map = map_array
        self.player = player
        self.enemy = enemy
        self.map_i = len(map_array)
        self.map_j = len(map_array[0])
        self.p_pos = player.location
        self.gen_map_grid()
        self.enemy_locations()
    def enemy_locations(self):
        self.enemy_locations =  [[i,j] for i in range(self.map_i) for j in range(self.map_j) if self.map[i][j] == 'red']

    def nw_label(self):
        l1 = w.Label('_', layout = w.Layout(width = 'auto',height = 'auto'))                
        l1.style.background = self.map[self.p_pos[0]][self.p_pos[1]]
        l1.style.text_color = self.map[self.p_pos[0]][self.p_pos[1]]
        return l1
    def w_label(self):
        l2 = w.Label('_', layout = w.Layout(width = 'auto',height = 'auto'))                
        l2.style.background = 'white'
        l2.style.text_color = 'white'
        return l2
    def check_fight(self):
        if self.p_pos in self.enemy_locations:
            clear_output(wait = True)
            enemy = Enemy()
            fight = Fight(player = self.player, enemy = enemy, overworld = self)
            fight.draw_fight()
    def buttons(self):
        buttons = w.GridspecLayout(3,3, 
                                    grid_gap = '0px',
                                    height = f'{100*3}px',
                                    width = f'{100*3}px')
        b_up = w.Button(description = 'Up', layout = w.Layout(width = 'auto',height = 'auto'))
        b_down = w.Button(description = 'Down', layout = w.Layout(width = 'auto',height = 'auto'))
        b_left = w.Button(description = 'Left', layout = w.Layout(width = 'auto',height = 'auto'))
        b_right = w.Button(description = 'Right', layout = w.Layout(width = 'auto',height = 'auto'))
        def upb(Btn):
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.nw_label()
            self.p_pos[0] = max(0,self.p_pos[0]-1)
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.w_label()
            self.check_fight()
        def downb(Btn):
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.nw_label()
            self.p_pos[0] = min(self.map_i-1,self.p_pos[0]+1)
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.w_label()
            self.check_fight()
        def leftb(Btn):
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.nw_label()
            self.p_pos[1] = max(0,self.p_pos[1]-1)
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.w_label()
            self.check_fight()
        def rightb(Btn):
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.nw_label()
            self.p_pos[1] = min(self.map_j-1,self.p_pos[1]+1)
            self.map_grid.children[0][self.p_pos[0],self.p_pos[1]] = self.w_label()
            self.check_fight()
        b_up.on_click(upb)
        b_down.on_click(downb)
        b_left.on_click(leftb)
        b_right.on_click(rightb)
        buttons[0,1] = b_up
        buttons[1,0] = b_left
        buttons[1,2] = b_right
        buttons[2,1] = b_down
        return buttons
    def gen_map_grid(self):
        map_grid = w.GridspecLayout(n_rows = self.map_i,
                                    n_columns = self.map_j, 
                                    grid_gap = '0px',
                                    height = f'{50*self.map_i}px',
                                    width = f'{50*self.map_j}px'
                                    )
        for i in range(self.map_i):
            for j in range(self.map_j):
                l1 = w.Label('_', layout = w.Layout(width = 'auto',height = 'auto'))                
                if [i,j] == self.p_pos:
                    l1.style.background = 'white'
                    l1.style.text_color = 'white'
                else:
                    l1.style.background = self.map[i][j]
                    l1.style.text_color = self.map[i][j]
                map_grid[i,j] = l1
        self.map_grid = w.HBox([map_grid,self.buttons()])
    def draw_map(self):
        
        display(self.map_grid)#w.HBox([self.map_grid,self.buttons()]))

class Fight():
    def __init__(self,
                 player,
                 enemy,
                 overworld):
        self.turn_count = 0
        self.create_gui()
        self.fighting = True
    def create_gui(self):
        p_hp_label = w.Label(str(int(player.hp_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-start"))
        p_mana_label = w.Label(str(int(player.mana_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-start"))

        e_hp_label = enemy.hp_label
        e_mana_label = enemy.mana_label
        
        p_box = w.VBox([
            w.HBox([p_hp_label,player.hp_bar]),
            w.HBox([p_mana_label,player.mana_bar])
        ])
        e_box = w.VBox([
            w.HBox([enemy.hp_bar,e_hp_label]),
            w.HBox([enemy.mana_bar,e_mana_label])
        ])
        
        moves_box = w.VBox([])
        def btn_fn(Btn):
            moves_box.layout.visibility = "hidden"
            enemy.remove_hp(player.attacks[Btn.description.split(' ')[0].lower()]['power'])
            if enemy.hp_bar.value == 0:
                self.fighting = False
                time.sleep(3)
                self.fight_box.close()
                l1 = w.Label('Fight Won!')
                display(l1)
                time.sleep(3)
                l1.close()
                overworld.draw_map()
            time.sleep(0.25)
            moves_box.layout.visibility = "visible"
            
        for move in list(player.attacks.keys()):
            
            b1 = w.Button(description = f"{move.capitalize()} {player.attacks[move]['power']} {player.attacks[move]['uses_left']}/{player.attacks[move]['max_uses']}")
            b1.on_click(btn_fn)
            
            moves_box.children=tuple(list(moves_box.children) + [b1])
            
        fight_box = w.GridspecLayout(2,3)
        
        fight_box[0,0] = p_box
        fight_box[0,1] = e_box
        fight_box[1,0] = moves_box
        
        self.fight_box = fight_box
    def draw_fight(self):
        display(self.fight_box)
        
        

In [ ]:
def main():
    global player
    player = Character()
    enemy = Enemy()
    
    test_t = w.BoundedIntText(value = 25,min = 0, max = 200)
    test_b = w.Button(description = 'Damage')
    p_hp_label = w.Label(str(int(player.hp_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-start"))
    p_mana_label = w.Label(str(int(player.mana_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-start"))
    
    e_hp_label = w.Label(str(int(enemy.hp_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-end"))
    e_mana_label = w.Label(str(int(enemy.mana_bar.value)), layout =w.Layout(width = '30px',display="flex", justify_content="flex-end"))
    
    hp_h = w.HBox([p_hp_label,player.hp_bar,enemy.hp_bar,e_hp_label], layout = w.Layout(width = '1600px'))
    mana_h = w.HBox([p_mana_label,player.mana_bar,enemy.mana_bar,e_mana_label],layout = w.Layout(width = '1600px'))
    
    reset_b = w.Button(description = 'Reset')
    def rstfn(Btn):
        clear_output(wait = True)
        main()
    reset_b.on_click(rstfn)
    
    def btnfn(Btn):
        
        rng = random.random()
        test_b.layout.visibility = "hidden"
        if rng > 0.5:
            player.remove_hp(test_t.value)
            p_hp_label.value = str(int(player.hp_bar.value))
        else:
            enemy.remove_hp(test_t.value)
            e_hp_label.value = str(int(enemy.hp_bar.value))
        if player.hp_bar.value ==0:
            time.sleep(1)
            out.close()
            print('Fainted')
            display(reset_b)
        elif enemy.hp_bar.value ==0:
            time.sleep(1)
            out.close()
            print('Enemy Defeated')
            display(reset_b)
        
        time.sleep(0.5)
        test_b.layout.visibility = "visible"
 
    test_b.on_click(btnfn)
    
    out = w.VBox([w.HBox([test_t,test_b]),w.VBox([hp_h,mana_h])])
    display(out)
    

        

In [ ]:
r = 'red'
g = 'green'
b = 'blue'
_ = 'black'
test_l = [g,b,_]
test_i = 10
test_j = 10
test_map = [[random.choice(test_l)for i in range(test_i)] for j in range(test_j)]
test_map[random.randint(0,test_i-1)][random.randint(0,test_j-1)] = r
player = Character()
overworld = Overworld(map_array=test_map,player = Character())#player_position=[int(test_i/2),int(test_j/2)])
overworld.draw_map()